<font color='black'>
<h1>
<span style="font-family:verdana; font-size:1.4em;">
Spark Streaming with Time Windows
</span>
</h1>
</font>

<font color='steelblue'>
<h2>
<span style="font-family:verdana; font-size:1.4em;">
Examples on Structured Streaming with Time Windows<br>
</span>
</h2>
</font>

<font color='grey'>
<span style="font-family:verdana; font-size:1.2em;">

Consists of examples of using windows on streaming data.<br>

</span>
<span style="font-family:verdana; font-size:1.2em;"><br>
    <b>Following is included in this notebook</b>:
    <ul>
        <li>Creating a read stream to receive parquet data from a certain folder</li>
        <li>Create a aggregate dataframe with a window of 1 hour</li>
        <li><b>Databricks only</b> Explore the aggregate dataframe and plot data</li>
        <li><b>Non Databricks:</b> Create in-memory query to explore the aggregate dataframe</li>
        <li>Explore performance impact of partitions</li>
        <li>Use Watermark to efficiently manage never ending streams</li>
    </ul>
</span>
</font>

<font color='tomato'>
<span style="font-family:verdana; font-size:1.2em;">
    <b>RUN ON DATABRICKS ONLY</b>:
</span>
</font>

In [ ]:
DATA_BRICKS = False

In [ ]:
if DATA_BRICKS == False:
    # Set up the environment for using pyspark
    import findspark
    findspark.init()
    
    from pyspark.sql import SparkSession
    from pyspark import SparkConf, SparkContext
    
    spark = SparkSession.builder.appName("Streaming Window").getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("Error")

In [ ]:
# Data type definitions, # Lot of functions used in this notebook
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Utility Functions

In [ ]:
def showDataFrame(df, numrecs = 5):
    if DATA_BRICKS == False:
        df.limit(numrecs).show(truncate = False)
    else:
        display(df)

def listFiles(dirname):
    if DATA_BRICKS == False:
      import os
      ret = os.listdir(dirname)
      for f in ret:
        print(f)
    else:
      ret = dbutils.fs.ls(dirname)
      print("Number of files {} Filenames are:".format(len(ret)))
      for f in ret:
        print(f.name)

def peekFile(fname, ftype = "parquet"):
    if DATA_BRICKS == False:
        if ftype == "parquet":
            tempdf = spark.read.parquet(fname)
            showDataFrame(tempdf)
        elif ftype == "json":
            tempdf = spark.read.json(fname)
            showDataFrame(tempdf)
        elif ftype == "csv":
            tempdf = spark.read.csv(fname)
            showDataFrame(tempdf)
        else:
            raise ValueError("invalid file type")

## Streaming Related Utility Functions

In [ ]:
# Get active streams
def getActiveStreams():
    try:
        return spark.streams.active
    except:
        # In extream cases, this funtion may throw an ignorable error.
        print("Unable to iterate over all active streams - using an empty set instead.")
    return []

# Stop a stream (passin stream object)
def stopStream(s):
    try:
        print("Stopping the stream {}.".format(s.name))
        s.stop()
        print("The stream {} was stopped.".format(s.name))
    except:
        # In extream cases, this funtion may throw an ignorable error.
        print("An [ignorable] error has occured while stoping the stream.")
        
# Stop all active streams
def stopAllStreams():
    streams = getActiveStreams()
    while len(streams) > 0:
        stopStream(streams[0])
        streams = getActiveStreams()
        
def printActiveStreams():
    streams = getActiveStreams()
    print(len(streams))
    for stream in streams:
        print(stream.name, stream.id)

# function to wait until a stream is ready to read
def untilStreamIsReady(name, progressions=3):
    import time
    queries = list(filter(lambda query: query.name == name, getActiveStreams()))

    while (len(queries) == 0 or len(queries[0].recentProgress) < progressions):
        time.sleep(5) # Give it a couple of seconds
        queries = list(filter(lambda query: query.name == name, getActiveStreams()))

    print("The stream {} is active and ready.".format(name))

def monitorMemStream(streamName, sqlStr):
    if DATA_BRICKS == False:
        import time
        for rec in range(4):
            spark.sql(sqlStr + memStream).show()
            time.sleep(4)

<font color='gray'>

<span style="font-family:verdana; font-size:1.4em;">
    <h3>
    Aggregating Streaming Data<br>
       </h3>
 </span>
<span style="font-family:verdana; font-size:1.2em;">
Many applications require real-time decisions done using aggregated statistics. e.g. aggregating errors in data streaming from IoT devices.<br>
Cannot aggregate on the whole datatset, because since this is streaming data there is no end - data will keep coming.
    </span>
 </font>

<font color='gray'>

<span style="font-family:verdana; font-size:1.4em;">
    <h3>
    Dataset for Windowed Streaming<br>
       </h3>
 </span>
<span style="font-family:verdana; font-size:1.2em;">
Dataset consists of a number of parquet files. Each record consits of `action` and `time`<br>
Imagine that these files are constantly being written to the folder and they are continuously streamed to our program<br>
First lets take a file and explore it.
</span>
</font>
<br>

In [ ]:
if DATA_BRICKS == False:
    path = "../datasets/TimeSeries/"
else:
    path = "dbfs:/FileStore/datasets/TimeSeries/"

pfile = "data0.parquet"

In [ ]:
peekFile(path + pfile)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.4em;">
    Creating a Read Stream and a aggregation dataframe
     
<br>
</span>
    </h3>
<span style="font-family:verdana; font-size:1.2em;">
    Following steps are performed:<br>
    <ul>
        <li>Define the schema</li>
        <li>Create a read stream for the folder containing the parquet files</li>
        <li>Create an aggregation dataframe based on this streaming data</li>
    </ul>
</span>
</font>

## Define the schema - Use DL formatted string

In [ ]:
sSchema = "action string, time timestamp"

## Create Read Stream<br>
<font color='gray'>
<span style="font-family:verdana; font-size:1.2em;">
    Using <i>maxFilesPerTrigger</i>, define how many files should receiver get (1 file per trigger).<br> Also define the file format.
<span>
<font>

In [ ]:
df = (spark
      .readStream                            # Returns DataStreamReader
      .option("maxFilesPerTrigger", 1)       # Force processing of only 1 file per trigger 
      .schema(sSchema)                       # Required for all streaming DataFrames
      .parquet(path)                         # The stream's source directory and file type
     )

In [ ]:
# Create a stream name
streamName = "windowStream"

## Create the aggregation dataframe using the streaming data

In [ ]:
countsDF = (df
            .groupBy(col("action"),                     # Aggregate by action...
                     window(col("time"), "1 hour"))     # ...then by a 1 hour window
            .count()                                    # For the aggregate, produce a count
            .select(col("window.start").alias("start"), # Elevate field to column
                    col("count"),                       # Include count
                    col("action"))                      # Include action
            .orderBy(col("start"))                      # Sort by the start time
           )

In [ ]:
memStream = "memory"
if DATA_BRICKS == False:
    memQuery = (countsDF
                .writeStream
                .format("memory")
                .queryName(memStream)
                .outputMode('complete')
                .start()
               )

## Check if the dataframe is streaming or static

In [ ]:
print("Is Streaming Dataframe: {}".format(countsDF.isStreaming))

In [ ]:
if DATA_BRICKS == True:
    display(countsDF, streamName = streamName)

In [ ]:
if DATA_BRICKS == True:
    untilStreamIsReady(streamName)

In [ ]:
printActiveStreams()

In [ ]:
if DATA_BRICKS == False:
    sqlStr = "select action, count from "
    monitorMemStream(memStream, sqlStr)

## Stream takes a while to get going - why?<br>
<font color='gray'>
<span style="font-family:verdana; font-size:1.2em;">
    Since in our aggregation there is a <i>`groupBy()`</i> being called which causes a <i>`shuffle`</i>. By default the <i>`Spark SQL has 200 partitions`</i>. Since we are doing stateful aggregation here means that there is a state map maintained for each partition. For fault tolerance purposes these maps are cached to the disk - which causes a lot of overhead.<br><br>
    <b>One way to enhance this performace is to reduce the number of partitions.</b>
<span>
<font>

## Check and set number of partitions

In [ ]:
print(spark.conf.get("spark.sql.shuffle.partitions"))

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

In [ ]:
print(spark.conf.get("spark.sql.shuffle.partitions"))

In [ ]:
pStream = "partStream"

In [ ]:
if DATA_BRICKS == True:
    display(countsDF,  streamName = pStream)

In [ ]:
if DATA_BRICKS == False:
    sqlStr = "select action, count from "
    monitorMemStream(memStream, sqlStr)

In [ ]:
if DATA_BRICKS == True:
    untilStreamIsReady(streamName)

In [ ]:
if DATA_BRICKS == True:
    stopAllStreams()

## Problem with generating many windows<br>
<font color='gray'>
<span style="font-family:verdana; font-size:1.2em;">
In the above code, a window is created for every 1 hour aggregate which is persisted and maintained. Over a period of time this aggregated data will build up in the driver - leading to performance issues and OOM error<br>
    Well, increase the window to <i>2 hours</i>, it may help but eventually the same issues will arise.
<span>
<font>

## Watermarking<br>
<font color='gray'>
<span style="font-family:verdana; font-size:1.2em;">
In watermarking, the stream is allowed to throw away the windows after certain cut-off.<br>
The following code sets up watermark:
    <ul>
        <li>Data received 2 hour past the watermark will be dropped</li>
        <li>Data received within the 2 hours of the watermark will never be dropped, meaning it will be aggregated</li>
        <li>Data delayed for more than 2 hours may not be dropped and also there is no guarantee that it will be aggregated</li>
<span>
<font>

In [ ]:
watermarkedDF = (df
                 .withWatermark("time", "2 hours")             # Specify a 2-hour watermark
                 .groupBy(col("action"),                       # Aggregate by action...
                          window(col("time"), "1 hour"))       # ...then by a 1 hour window
                 .count()                                      # For each aggregate, produce a count
                 .select(col("window.start").alias("start"),   # Elevate field to column
                         col("count"),                         # Include count
                         col("action"))                        # Include action
                 .orderBy(col("start"))                        # Sort by the start time
                )

In [ ]:
memStream1 = "memory1"
if DATA_BRICKS == False:
    memQuery1 = (watermarkedDF
                .writeStream
                .format("memory")
                .queryName(memStream1)
                .outputMode('complete')
                .start()
               )

In [ ]:
streamName = "watermark"
if DATA_BRICKS == True:
    display(watermarkedDF, streamName = streamName)

In [ ]:
if DATA_BRICKS == False:
    sqlStr = "select action, count from "
    monitorMemStream(memStream1, sqlStr)

In [ ]:
if DATA_BRICKS == True:
    untilStreamIsReady(streamName)

In [ ]:
stopAllStreams()